In [ ]:
import pymysql
import requests
import json

In [ ]:
passes = open("pass.txt", "r").read()
passes = passes.split("\n")

### Take in text document and clean info to get student usernames and login credentials. w

In [ ]:
stud = passes[::2]
new_stud = []
for student in stud:
    student = student.split('"')
    try:
        new_stud.append(student[1])
    except:
        pass
passw = passes[1::2]
new_pass = []
for line in passw:
        new_line = line.split(" ")
        port = int(new_line[4])
        passw = f'{new_line[7][2:]}'
        creds = [new_line[2], port, passw]
        new_pass.append(creds)   

### Create students dictionary from edited information.

In [ ]:
students = dict(zip(new_stud, new_pass)) 

### Get counts of querys students' midterm query logs. 

In [ ]:
#connect to MYSQL server
for key, val in students.items():
    try:
        con = pymysql.connect(host = val[0], port = val[1], user = 'root', password = val[2], database= 'mysql',                    cursorclass=pymysql.cursors.DictCursor)
        print(f"connected to {key}'s database")
        cur = con.cursor()
        print("constructing query:")
        cur.execute(f'select count(1) as Exam_Query_Count from general_log where user_host like "%{key}%"and command_type = "Query";')
        queries = cur.fetchall()
        print(queries)
        doc = open("midterm_query_counts.txt", "a+")
        doc.write(f"{key}, {queries}\n")
        con.close()
    except:
        doc = open("midterm_query_counts.txt", "a+")
        doc.write(f"{key}, could not open database\n")
        print("encountered an ERROR")
        print(key, val)

### For student in suspect list, find full query log. 

In [ ]:
suspects = []


for key, val in students.items():
    if key in final_suspect:
        con = pymysql.connect(host = val[0], port = val[1], user = 'root', password = val[2], database= 'mysql',                 cursorclass=pymysql.cursors.DictCursor)
        print(f"connected to {key}'s database")
        cur = con.cursor()
        print("constructing query:")
        cur.execute(f'select event_time, argument as query from general_log where user_host like "%{key}%"and command_type = "Query" and day(event_time) in (02, 03);')
        for row in cur:
            query = f"{row['event_time']}, {row['query']}"
            doc = open("midterm_query_finalsuspect.txt", "a+")
            doc.write(f"{key}, {query}\n")
        con.close()

    else:
        pass